In [1]:
#import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

from time import sleep
from random import randint


In [7]:
# Scrape the desired url, only one page

url = 'http://www.guide2research.com/topconf/machine-learning'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

### ==== some inspection to understand the html file structure ====

In [11]:
soup.find_all("table")[0:4]

[<table border="0" cellpadding="0" cellspacing="0" style="margin:1px;padding:1px;" width="100%">
 <tr>
 <td style="padding:1px;margin:0px;vertical-align:middle" width="35px">
 <center><i style="font-size:14px;"></i></center>
 </td>
 <td style="padding:1px;margin:0px;vertical-align:middle" width="65px">
 <center><i style="font-size:14px;"><i>H5-index</i></i></center>
 </td>
 <td style="padding:1px;margin:0px;vertical-align:middle" width="115px">
 <center><i style="font-size:14px;"><i>Publisher</i></i></center>
 </td>
 <td style="margin:1px;padding:1px;vertical-align:middle;">
 <center><i style="font-size:14px;"><i>Conference Details</i></i></center>
 </td>
 </tr>
 </table>,
 <table border="0" cellpadding="0" cellspacing="0" style="margin:1px;padding:1px;" width="100%">
 <tr>
 <td rowspan="2" style="padding:1px;margin:0px;vertical-align:middle" width="35px">
 <center><i style="font-size:18px;"></i></center>
 </td>
 <td rowspan="2" style="padding:1px;margin:0px;vertical-align:middle" widt

In [13]:
context = soup.find("div", {"id": "ajax_content"})

In [15]:
tables = context.find_all("table")

In [17]:
print(len(tables))
table_test = tables[0]

90


In [23]:
table_test

<table border="0" cellpadding="0" cellspacing="0" style="margin:1px;padding:1px;" width="100%">
<tr>
<td rowspan="2" style="padding:1px;margin:0px;vertical-align:middle" width="35px">
<center><i style="font-size:18px;"></i></center>
</td>
<td rowspan="2" style="padding:1px;margin:0px;vertical-align:middle" width="65px">
<center><b style="font-size:22px;">240</b></center>
</td>
<td rowspan="2" style="padding:1px;margin:0px;vertical-align:middle" width="115px">
<img src="/img/IEEE_s.png" style="vertical-align:middle;width:110px;"/>
</td>
<td colspan="3" style="margin:1px;padding:1px;vertical-align:middle;"><h4 style="padding:1px;margin:1px;font-size:14px;"><a href="/?p=13992">
CVPR : IEEE/CVF Conference on Computer Vision and Pattern Recognition</a>
</h4>
</td>
</tr>
<tr>
<td style="padding:1px;margin:0px;vertical-align:middle;" valign="top" width="400px">
<div style="padding:0px;margin:0px;">Jun 16, 2020 - Jun 18, 2020 - Seattle , <b>United States</b></div>
<div style="padding:0px;margi

In [19]:
row = table_test.find_all("tr")

In [33]:
row

[<tr>
 <td rowspan="2" style="padding:1px;margin:0px;vertical-align:middle" width="35px">
 <center><i style="font-size:18px;"></i></center>
 </td>
 <td rowspan="2" style="padding:1px;margin:0px;vertical-align:middle" width="65px">
 <center><b style="font-size:22px;">240</b></center>
 </td>
 <td rowspan="2" style="padding:1px;margin:0px;vertical-align:middle" width="115px">
 <img src="/img/IEEE_s.png" style="vertical-align:middle;width:110px;"/>
 </td>
 <td colspan="3" style="margin:1px;padding:1px;vertical-align:middle;"><h4 style="padding:1px;margin:1px;font-size:14px;"><a href="/?p=13992">
 CVPR : IEEE/CVF Conference on Computer Vision and Pattern Recognition</a>
 </h4>
 </td>
 </tr>,
 <tr>
 <td style="padding:1px;margin:0px;vertical-align:middle;" valign="top" width="400px">
 <div style="padding:0px;margin:0px;">Jun 16, 2020 - Jun 18, 2020 - Seattle , <b>United States</b></div>
 <div style="padding:0px;margin:0px;font-size:11px;line-height:11px;"><a href="http://cvpr2020.thecvf.com/

In [42]:
row[0].find_all("td")[1].get_text()

'\n240\n'

In [28]:
# the first element is about the name of the journal;
# the second is the website
row[0].find_all("td")[3].get_text()

'\nCVPR : IEEE/CVF Conference on Computer Vision and Pattern Recognition\n\n'

In [35]:
import re
re.sub('\n', '', row[0].find_all("td")[3].get_text())

'CVPR : IEEE/CVF Conference on Computer Vision and Pattern Recognition'

In [32]:
row[1].find_all("td")[0].find("a")["href"]

'http://cvpr2020.thecvf.com/'

### ==== some inspection to understand the html file structure (END) ====

In [43]:
# put it into a code block

context = soup.find("div", {"id": "ajax_content"})
tables = context.find_all("table")

rows = []

for table in tables:
    # each row is the info about the conference
    row = table.find_all("tr")
    # get the h5 index, need to remove the newline. e.g., '\n240\n'
    h5_index = re.sub('\n', '', row[0].find_all("td")[1].get_text())
    # the name of the conference, remove the "newline"
    name = re.sub('\n', '', row[0].find_all("td")[3].get_text())
    # the website
    url = row[1].find_all("td")[0].find("a")["href"]
    
    rows.append([h5_index, name, url])

In [4]:
#print(page.status_code)

In [44]:
df=pd.DataFrame(rows,columns=['h5_index', 'Name','URL'])

In [45]:
df.head(10)

,h5_index,Name,URL
0,240,CVPR : IEEE/CVF Conference on Computer Vision ...,http://cvpr2020.thecvf.com/
1,169,NeurIPS : Neural Information Processing System...,https://nips.cc/Conferences/2020/CallForPapers
2,137,ECCV : European Conference on Computer Vision,https://eccv2020.eu/
3,135,ICML : International Conference on Machine Lea...,https://icml.cc/Conferences/2020
4,129,ICCV : IEEE/CVF International Conference on Co...,http://iccv2021.thecvf.com/home
5,106,ACL : Meeting of the Association for Computati...,https://2021.aclweb.org/
6,95,AAAI : AAAI Conference on Artificial Intelligence,https://aaai.org/Conferences/AAAI-21/
7,88,EMNLP : Conference on Empirical Methods in Nat...,https://2020.emnlp.org/
8,86,SIGKDD : ACM SIGKDD International Conference o...,https://www.kdd.org/kdd2020/
9,67,IJCAI : International Joint Conference on Arti...,https://ijcai20.org


In [49]:
df["h5_index"] = df["h5_index"].apply(lambda x: int(x))

In [50]:
df.dtypes

h5_index     int64
Name        object
URL         object
dtype: object

In [52]:
df.to_csv('top_conference_ML_AI.csv',header=True)